In [13]:
import requests
from bs4 import BeautifulSoup
import mysql.connector

In [14]:
from datetime import datetime
import concurrent.futures

# List of days to check
days_to_check = [1, 2, 3, 15, 16, 17, 30, 31]

# List of months to check
months_to_check = range(1, 13)

# Range of years to check
year_start = 2006
year_end = 2024

# Generate all specified dates in the range of years
available_dates = []

def check_date(year, month, day):
	try:
		date = datetime(year + 543, month, day)
		test_url = f"https://news.sanook.com/lotto/check/{date.strftime('%d%m')}{date.strftime('%Y')}/"
		response = requests.get(test_url)
		if response.status_code == 200:
			soup = BeautifulSoup(response.content, 'html.parser')
			div_r5 = soup.find('div', class_='lottocheck__sec lottocheck__sec--font-mini')
			if div_r5:
				numbers_r5 = [span.text for span in div_r5.find_all('span', class_='lotto__number')]
				if numbers_r5:
					available_dates.append(date.strftime("%Y-%m-%d"))
	except ValueError:
		# Skip invalid dates (e.g., April 31)
		pass

with concurrent.futures.ThreadPoolExecutor() as executor:
	futures = []
	for year in range(year_start, year_end + 1):
		for month in months_to_check:
			for day in days_to_check:
				futures.append(executor.submit(check_date, year, month, day))
	concurrent.futures.wait(futures)

print(available_dates)

['2549-12-30', '2550-01-16', '2550-02-01', '2550-02-16', '2550-03-16', '2550-03-01', '2550-04-16', '2550-04-01', '2550-05-02', '2550-05-16', '2550-06-01', '2550-06-16', '2550-07-16', '2550-07-01', '2550-08-01', '2550-08-16', '2550-09-01', '2550-09-16', '2550-10-01', '2550-11-01', '2550-10-16', '2550-11-16', '2550-12-16', '2550-12-30', '2551-02-01', '2551-01-16', '2551-03-01', '2551-03-16', '2551-02-16', '2551-04-01', '2551-04-16', '2551-05-02', '2551-05-16', '2551-06-16', '2551-07-01', '2551-06-01', '2551-07-16', '2551-08-16', '2551-08-01', '2551-09-01', '2551-09-16', '2551-10-16', '2551-10-01', '2551-11-01', '2551-12-16', '2551-12-01', '2551-11-16', '2551-12-30', '2552-01-16', '2552-02-01', '2552-02-16', '2552-04-01', '2552-03-01', '2552-04-16', '2552-05-02', '2552-05-16', '2552-03-16', '2552-06-01', '2552-06-16', '2552-07-01', '2552-07-16', '2552-08-01', '2552-08-16', '2552-09-01', '2552-10-01', '2552-09-16', '2552-11-01', '2552-11-16', '2552-12-01', '2552-10-16', '2552-12-16', '2552

In [15]:
import pandas as pd
from datetime import datetime
import mysql.connector
import requests
from bs4 import BeautifulSoup

# Connect to the MySQL database
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="JamesDominolity1903",
    database="lottery"
)

cursor = conn.cursor()

# Define the select and insert queries
select_query = "SELECT COUNT(*) FROM lottery WHERE date = %s AND reward_type = %s"
insert_query = "INSERT INTO lottery (date, reward_type, number) VALUES (%s, %s, %s)"

# List of available dates (example)
reward_type = "5"

# Function to convert Buddhist year to Gregorian year
def convert_buddhist_to_gregorian(date_str):
    buddhist_date = datetime.strptime(date_str, '%Y-%m-%d')
    gregorian_year = buddhist_date.year - 543
    gregorian_date = buddhist_date.replace(year=gregorian_year)
    return gregorian_date.strftime('%Y-%m-%d')


# Use a session to reuse the connection
with requests.Session() as session:
    for date_str in available_dates:
        # Convert the date string to datetime object
        buddhist_date = datetime.strptime(date_str, '%Y-%m-%d')

        # Generate the URL for the specific date using Buddhist year
        test_url = f"https://news.sanook.com/lotto/check/{buddhist_date.strftime('%d%m')}{buddhist_date.strftime('%Y')}/"

        # Fetch the URL content
        response = session.get(test_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find the div containing the numbers for รางวัลที่ 5
            div_r5 = soup.find(
                'div', class_='lottocheck__sec lottocheck__sec--font-mini')

            if div_r5:
                # Extract all the numbers within that div
                numbers_r5 = [span.text for span in div_r5.find_all(
                    'span', class_='lotto__number')]

                # Convert the date string to Gregorian date
                gregorian_date_str = convert_buddhist_to_gregorian(date_str)

                # Check if the date already exists in the database
                cursor.execute(select_query, (gregorian_date_str, reward_type))
                result = cursor.fetchone()

                if result[0] == 0:
                    # Insert each number into the database
                    for number in numbers_r5:
                        cursor.execute(
                            insert_query, (gregorian_date_str, reward_type, number))
                else:
                    print(
                        f"Date {gregorian_date_str} already exists in the database")
            else:
                print(f"No div found for date: {date_str}")
        else:
            print(f"Failed to fetch URL for date: {date_str}")

# Commit the transaction once at the end
conn.commit()

# Close the database connection
conn.close()

print("Dates have been updated to Gregorian format in the database and saved to lottery_data.xlsx")

Dates have been updated to Gregorian format in the database and saved to lottery_data.xlsx


In [24]:
import mysql.connector
import pandas as pd

# Re-establish the connection to the MySQL database
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="JamesDominolity1903",
    database="lottery"
)

# Fetch the data from the database
query = "SELECT * FROM lottery"
df = pd.read_sql(query, conn)

# Inspect the first few rows of the DataFrame
print(df.head())

# Check the data type of the 'date' column
print(df['date'].dtype)

# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

# Verify the conversion
print(df['date'].head())

# Ensure the date format is yyyy-mm-dd when exporting to Excel
df['date'] = df['date'].dt.strftime('%Y-%m-%d')

# Save the DataFrame to an Excel file
df.to_excel("lottery_data.xlsx", index=False)

print("Data has been saved to lottery_data.xlsx")

# Close the database connection
conn.close()

C:\Users\Kolawat\AppData\Local\Temp\ipykernel_22588\3456019144.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


       id        date  reward_type  number
0  191601  2006-12-30            5  004495
1  191602  2006-12-30            5  034106
2  191603  2006-12-30            5  049904
3  191604  2006-12-30            5  067124
4  191605  2006-12-30            5  069215
object
0   2006-12-30
1   2006-12-30
2   2006-12-30
3   2006-12-30
4   2006-12-30
Name: date, dtype: datetime64[ns]
Data has been saved to lottery_data.xlsx
